In [ ]:
! pip install --upgrade boto3 sagemaker

In [ ]:

# Define IAM role
import boto3
import re
import requests
import base64
import json
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import cv2
import base64

role = get_execution_role()
print(role)

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/inference/mytriton:latest'.format(account, region)
print(image)


In [ ]:
#for triron
container = {
    "Image": image,
    "Environment": {"SAGEMAKER_TRITON_DEFAULT_MODEL_NAME": "resnet"},
}

model = sess.create_model(
    name='mytriton', role=role, container_defs=container
)
print(model)

In [ ]:
endpoint_cfg=sess.create_endpoint_config(
        name="MYTRITONCFG",
        model_name="mytriton",
        initial_instance_count=1,
        instance_type="ml.g4dn.2xlarge"
    )

In [ ]:
endpoint=sess.create_endpoint(
        endpoint_name="MyTritonEndpoint", config_name="MYTRITONCFG")
print(endpoint)

In [ ]:
from IPython.display import Image
Image(url= "dog.jpg",width=100, height=100)

In [ ]:
img = cv2.imread('dog.jpg')
string_img = base64.b64encode(cv2.imencode('.jpg', img)[1]).decode()

In [ ]:
## restnet client
runtime = boto3.Session().client('runtime.sagemaker')

payload = json.dumps({"modelname": "resnet","payload": {"img":string_img}})

endpoint="MyTritonEndpoint"
response = runtime.invoke_endpoint(EndpointName=endpoint,ContentType="application/json",Body=payload,Accept='application/json')

out=response['Body'].read()


In [ ]:
res=eval(out)
print(res)

In [ ]:
# yolov5 client
payload = json.dumps({"modelname": "yolov5","payload": {"img":string_img}})

endpoint="MyTritonEndpoint"
response = runtime.invoke_endpoint(EndpointName=endpoint,ContentType="application/json",Body=payload,Accept='application/json')

out=response['Body'].read()


In [ ]:
res=eval(out)
print(str(out))

In [ ]:
# bert client
text="The capital of China is [MASK]."
text="The biggest city of China is [MASK]."
text="The world has [MASK] people."

payload = json.dumps({"modelname": "bert_base","payload": {"text":text}})

endpoint="MyTritonEndpoint"
response = runtime.invoke_endpoint(EndpointName=endpoint,ContentType="application/json",Body=payload,Accept='application/json')

out=response['Body'].read()
res=eval(out)
print(res)

In [ ]:
#Delete the endpoint
boto3.client('sagemaker').delete_endpoint(EndpointName=endpoint)